## 낙상 사고 감지 데이터셋

해당 데이터 셋에서 다루는 주요 문제는 이벤트를 정상적인 생활 활동 또는 낙상 이벤트로 분류하는 문제이다. 목표는 가슴, 발목, 벨트와 같은 사람의 몸의 다양한 위치에서 수집한 센서 데이터를 기반으로 낙상 사례를 정확하게 식별할 수 있는 알고리즘을 개발하는 것이다.

실시간으로 낙상을 감지하면 적절한 조치를 취하여 사람에게 적시에 도움을 제공하여 잠재적으로 생명을 구하고 낙상과 관련된 위험을 줄일 수 있다.

### 필요 라이브러리 Import

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Dense
from keras.metrics import mae

In [ ]:
def concat_csv_files(root_path):
    filenames = os.listdir(root_path)

    data = pd.DataFrame()
    for filename in filenames:
        csv_path = os.path.join(root_path, filename)
        data_frame = pd.read_csv(csv_path)
        data = pd.concat([data, data_frame]).reset_index(drop=True)
    
    return data

data = concat_csv_files("../data/fall_accident/train")

In [ ]:
data.to_csv("./dataset.csv", encoding="utf8", sep=",", index=False)

In [ ]:
data.info()

In [ ]:
features = data.iloc[:, :-1]
labels = data.iloc[:, -1]

In [ ]:
features

In [ ]:
labels

In [ ]:
fig = px.histogram(labels)
fig.update_layout(width=800, height=600)
fig.update_yaxes(tickformat=",")
fig.show()

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=705)

In [ ]:
train_data

In [ ]:
scaler = StandardScaler()

scaler.fit(train_data[['x', 'y', 'z']])

In [ ]:
train_data[['x', 'y', 'z']] = scaler.transform(train_data[['x', 'y', 'z']])
test_data[['x', 'y', 'z']] = scaler.transform(test_data[['x', 'y', 'z']])

In [ ]:
train_data = train_data.values
test_data = test_data.values

train_labels = train_labels.astype(bool).values
test_labels = test_labels.astype(bool).values

n_train_data = train_data[~train_labels]
n_test_data = test_data[~test_labels]

an_train_data = train_data[train_labels]
an_test_data = test_data[test_labels]

In [ ]:
n_train_data.shape

In [ ]:
from keras.layers import BatchNormalization, Activation

class detector(Model):
    def __init__(self):
        super(detector, self).__init__()
        self.encoder = tf.keras.Sequential([
            Dense(128),
            BatchNormalization(),
            Activation("silu"),
            Dense(64),
            BatchNormalization(),
            Activation("silu"),
            Dense(32),
            BatchNormalization(),
            Activation("silu"),
            Dense(16),
            BatchNormalization(),
            Activation("silu"),
            Dense(8),
        ])

        self.decoder = tf.keras.Sequential([
            Dense(16),
            BatchNormalization(),
            Activation("silu"),
            Dense(32),
            BatchNormalization(),
            Activation("silu"),
            Dense(64),
            BatchNormalization(),
            Activation("silu"),
            Dense(128),
            BatchNormalization(),
            Activation("silu"),
            Dense(7, activation="sigmoid"),
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)

        return decoded
    
autoencoder = detector()
autoencoder.compile(optimizer="adam", loss="mae")
autoencoder.fit(  # call "call method" in class "detector"
    n_train_data, 
    n_train_data, 
    epochs=300, 
    validation_data=(n_test_data, n_test_data), 
    batch_size=512,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.3, mode="min", patience=3),
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=5)
        ]
    )

In [ ]:
reconstructed = autoencoder(n_train_data)  # 해당 시점에서도 call 메소드 호출
train_loss = mae(reconstructed, n_train_data)  # auto encoder로 인해 reconstruct된 데이터와 훈련 데이터 간의 reconstruction error를 계산
t = np.mean(train_loss) + np.std(train_loss)  # reconstruction error의 평균과 표준편차의 합으로 threshold를 설정

def prediction(model, data, threshold):
    rec = model(data)
    loss = mae(rec, data)
    return tf.math.less(loss, threshold)

print(t)

In [ ]:
pred = prediction(autoencoder, n_test_data, t)
pred